# 제 2유형 연습하기 : 타이타닉 생존자 분류

## ✅ 데이터 분석 순서
1. 라이브러리 및 데이터 탐색
2. 데이터 탐색 (EDA)
3. 데이터 전처리 및 분리
4. 모델링 및 성능 평가
5. 예측값 제출

### ✅ 1. 라이브러리 및 데이터 탐색

In [195]:
import pandas as pd
import numpy as np

In [196]:
############### 복사 영역 ############### 
# # 실기 시험 데이터셋으로 셋팅하기 (수정금지)
# Seaborn의 내장 타이타닉 데이터셋을 불러옵니다. 
import seaborn as sns
df = sns.load_dataset('titanic')
x = df.drop('survived', axis=1) 
y = df['survived']
# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train) 
y_train = pd.DataFrame(y_train) 
y_test = pd.DataFrame(y_test) # 평가용
x_test.reset_index()
y_train.columns = ['target'] 
y_test.columns = ['target'] 
############### 복사 영역 ###############
### 참고사항 ###
# y_test 는 실기 문제상에 주어지지 않음
# ★Tip : X를 대문자로 쓰지말고 소문자 x로 쓰세요. 시험에서 실수하기 쉽습니다.(문제풀기 전에 소문자로 변경!) 
# # (참고 : 보통 X는 2차원 배열(행렬)이기 때문에 대문자로 쓰고, y는 1차원 배열(벡터)이기 때문에 소문자로 씀)
# (~23년 10월말) 실기시험 데이터 형식 (실제 시험장에서는 다를 수 있으니 반드시 체크) 
# # X_test = pd.read_csv("data/X_test.csv")
# X_train = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")
# ★(23년 10월말~) 기준으로 체험환경에서 제공되는 데이터셋이 조금 변경되었습니다. 
# # train = pd.read_csv("data/customer_train.csv")
# test = pd.read_csv("data/customer_test.csv")
# x_train과 y_train, x_test를 별도로 할당해주셔야 합니다.

타이타닉 생존자 예측 문제
- 데이터의 결측치, 중복 변수값에 대해 처리하고 
- 분류모델을 사용하여 Accuracy, F1 score, AUC 값을 산출하시오.

데이터 설명
> - survival : 0 = No, 1 = Yes
> - pclass : 객실 등급 (1, 2, 3)
> - sex  : 성별
> - age : 나이
> - sibsp : 타이타닉호에 탑승한 형제/배우자의 수
> - parch : 타이타닉호에 탑승한 부모/자녀의 수
> - fare : 요금
> - embarked : 탑승지 이름(C, Q, S : Cherbourg / Queenstown / Southampton)
> - (중복) class : 객실 등급 (First, Second, Third)
> - who : man, woman, child
> - adult_male : 성인남자인지 여부(True=성인 남자, False= 그외)
> - deck : 선실번호 첫 알파벳(A, B, C, D, E, F, G)
> - (중복) embark_town : 탑승지 이름(Cherbourg, Queenstown, Southampton) 
> - (중복) alive : 생존여부(no:사망, yes:생존)
> - alone : 혼자 탑승했는지 여부(True=혼자, False=가족과 함께)

### ✅ 2. 데이터 탐색 (EDA)

In [197]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(712, 14)
(179, 14)
(712, 1)


In [198]:
print(x_train.head())
print(x_test.head())
print(y_train.head())

     pclass     sex   age  sibsp  parch   fare embarked   class    who  \
3         1  female  35.0      1      0  53.10        S   First  woman   
517       3    male   NaN      0      0  24.15        Q   Third    man   
861       2    male  21.0      1      0  11.50        S  Second    man   
487       1    male  58.0      0      0  29.70        C   First    man   
58        2  female   5.0      1      2  27.75        S  Second  child   

     adult_male deck  embark_town alive  alone  
3         False    C  Southampton   yes  False  
517        True  NaN   Queenstown    no   True  
861        True  NaN  Southampton    no  False  
487        True    B    Cherbourg    no   True  
58        False  NaN  Southampton   yes  False  
     pclass     sex   age  sibsp  parch      fare embarked   class    who  \
800       2    male  34.0      0      0   13.0000        S  Second    man   
341       1  female  24.0      3      2  263.0000        S   First  woman   
413       2    male   NaN     

In [199]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   pclass       712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          579 non-null    float64 
 3   sibsp        712 non-null    int64   
 4   parch        712 non-null    int64   
 5   fare         712 non-null    float64 
 6   embarked     710 non-null    object  
 7   class        712 non-null    category
 8   who          712 non-null    object  
 9   adult_male   712 non-null    bool    
 10  deck         164 non-null    category
 11  embark_town  710 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(5)
memory usage: 64.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 14 columns):
 #   Column       No

In [200]:
# train, test 데이터의 기초통계량 비교 
print(x_train.describe().T)
print(x_test.describe().T)

        count       mean        std   min      25%      50%     75%       max
pclass  712.0   2.307584   0.834926  1.00   2.0000   3.0000   3.000    3.0000
age     579.0  29.479568  14.355304  0.42  20.0000  28.0000  38.000   74.0000
sibsp   712.0   0.518258   1.094522  0.00   0.0000   0.0000   1.000    8.0000
parch   712.0   0.372191   0.792341  0.00   0.0000   0.0000   0.000    6.0000
fare    712.0  31.741836  45.403910  0.00   7.8958  14.4542  31.275  512.3292
        count       mean        std  min     25%   50%      75%       max
pclass  179.0   2.312849   0.842950  1.0   2.000   3.0   3.0000    3.0000
age     135.0  30.640741  15.258427  1.0  22.000  29.0  39.0000   80.0000
sibsp   179.0   0.541899   1.137797  0.0   0.000   0.0   1.0000    8.0000
parch   179.0   0.418994   0.859760  0.0   0.000   0.0   0.0000    5.0000
fare    179.0  34.043364  64.097184  0.0   7.925  14.5  30.2854  512.3292


In [201]:
# object, category 데이터도 추가 확인
print(x_train.describe(include='object'))
print(x_test.describe(include='object'))

print(x_train.describe(include='category'))
print(x_test.describe(include='category'))

         sex embarked  who  embark_town alive
count    712      710  712          710   712
unique     2        3    3            3     2
top     male        S  man  Southampton    no
freq     469      518  432          518   439
         sex embarked  who  embark_town alive
count    179      179  179          179   179
unique     2        3    3            3     2
top     male        S  man  Southampton    no
freq     108      126  105          126   110
        class deck
count     712  164
unique      3    7
top     Third    C
freq      391   47
        class deck
count     179   39
unique      3    7
top     Third    C
freq      100   12


In [202]:
# target 데이터도 상세 확인
y_train.head()

,target
3,1
517,0
861,0
487,0
58,1


In [203]:
y_train.describe()

,target
count,712.000000
mean,0.383427
std,0.486563
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [204]:
print(y_train.value_counts())

target
0         439
1         273
Name: count, dtype: int64


### ✅ 3. 데이터 전처리 및 분리
#### 1) 결측치, 2) 이상치, 3) 변수 처리하기

In [205]:
# 결측치 확인 : isnull().sum()
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

pclass           0
sex              0
age            133
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           548
embark_town      2
alive            0
alone            0
dtype: int64
pclass           0
sex              0
age             44
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
deck           140
embark_town      0
alive            0
alone            0
dtype: int64
target    0
dtype: int64


In [206]:
# 결측치 제거
# df = df.dropna()
# print(df)

In [207]:
# 결측치 대체
# x_train(712, 14) : age(133), embarked(2), deck(548)
# x_test(179, 14) : age(44), deck(140)

# 변수 제거
# (중복) class
# (중복) embark_town
# (중복) alive
# (결측치 다수) deck

변수를 제거하고 결측치를 대체할지, 결측치를 대체하고 변수를 제거할 지는 내 선택..?

In [208]:
# 중복 변수 제거
x_train = x_train.drop(columns=['class', 'embark_town', 'alive', 'deck'])
x_test = x_test.drop(['class', 'embark_town', 'alive', 'deck'], axis=1)

In [209]:
# 변수 제거 확인
print(x_train.info())
print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      712 non-null    int64  
 1   sex         712 non-null    object 
 2   age         579 non-null    float64
 3   sibsp       712 non-null    int64  
 4   parch       712 non-null    int64  
 5   fare        712 non-null    float64
 6   embarked    710 non-null    object 
 7   who         712 non-null    object 
 8   adult_male  712 non-null    bool   
 9   alone       712 non-null    bool   
dtypes: bool(2), float64(2), int64(3), object(3)
memory usage: 51.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      179 non-null    int64  
 1   sex         179 non-null    object 
 2   age         135 non-null    float64
 3   sibsp       179 non

In [210]:
# 결측치 대체
# x_train(712, 14) : age(133), embarked(2)
# x_test(179, 14) : age(44)

# age 변수
median_age = x_train['age'].median()
x_train['age'] = x_train['age'].fillna(median_age)
x_test['age'] = x_test['age'].fillna(median_age)

# embarked 변수
mode_embarked = x_train['embarked'].mode()
x_train['embarked'] = x_train['embarked'].fillna(mode_embarked[0])

In [211]:
# 결측치 대체 여부 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())

pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
who           0
adult_male    0
alone         0
dtype: int64
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
who           0
adult_male    0
alone         0
dtype: int64


In [212]:
# 변수 처리 (원핫인코딩)
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

print(x_train.info())
print(x_test.info())

# 하이퍼파라미터 튜닝을 위한 advanced 버전 저장
x_train_ad = x_train.copy()
x_test_ad = x_test.copy()
y_train_ad = y_train.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      712 non-null    int64  
 1   age         712 non-null    float64
 2   sibsp       712 non-null    int64  
 3   parch       712 non-null    int64  
 4   fare        712 non-null    float64
 5   adult_male  712 non-null    bool   
 6   alone       712 non-null    bool   
 7   sex_female  712 non-null    bool   
 8   sex_male    712 non-null    bool   
 9   embarked_C  712 non-null    bool   
 10  embarked_Q  712 non-null    bool   
 11  embarked_S  712 non-null    bool   
 12  who_child   712 non-null    bool   
 13  who_man     712 non-null    bool   
 14  who_woman   712 non-null    bool   
dtypes: bool(10), float64(2), int64(3)
memory usage: 40.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dt

In [213]:
# (참고) 원핫인코딩 후 feature 컬럼 변수의 개수가 다른 경우
# x_train = pd.get_dummies(x_train)
# x_test = pd.get_dummies(x_test)
# x_train.info()
# x_test.info()

# 해결방법(x_test의 변수가 더 많은 경우의 코드)
# x_train = x_train.reindex(columns=x_test.columns, fill_value=0)
# x_train.info()

#### 데이터 분리

In [214]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (훈련용 80%, 검증용 20%)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'], test_size=0.2, stratify=y_train['target'], random_state=2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(569, 15)
(143, 15)
(569,)
(143,)


### ✅ 4. 모델링 및 성능 평가

In [215]:
# 랜덤포레스트 모델 사용
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2023)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=2023)

In [216]:
# 예측
y_pred = model.predict(x_val)

In [217]:
# accuracy, f1 score, auc 이용한 성능 평가
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred)

In [218]:
print(acc)
print(f1)
print(auc)

0.8531468531468531
0.8108108108108109
0.8465909090909092


##### 실제 test 셋으로 성능평가를 한다면?

In [219]:
y_pred_f = model.predict(x_test)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
acc_f = accuracy_score(y_test, y_pred_f)
f1_f = f1_score(y_test, y_pred_f)
auc_f = roc_auc_score(y_test, y_pred_f)

In [220]:
print(acc_f)
print(f1_f)
print(auc_f)

0.7821229050279329
0.7153284671532847
0.7687088274044797


In [221]:
# 하이퍼파라미터 튜닝 : 37분 ~ 41분

### ✅ 5. 예측값 제출

In [222]:
# (실기시험 안내사항)
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({ 'result': y_result }).to_csv('수험번호.csv', index=False)
# 모델을 사용하여 테스트 데이터 예측

# 1. 특정 클래스로 분류할 경우 : predict
y_result = model.predict(x_test)
print(y_result[:5])

# 2. 특정 클래스로 분류될 확률을 계산하는 경우 : predict_proba()
y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

# 이해해보기
df = pd.DataFrame({'result' : y_result, 'prob_0 (class_0)' : y_result_prob[:, 0], 'prob_1 (class_1)' : y_result_prob[:, 1]})
print(df.head())

[1 1 0 0 0]
[[0.32 0.68]
 [0.24 0.76]
 [1.   0.  ]
 [0.93 0.07]
 [0.93 0.07]]
   result  prob_0 (class_0)  prob_1 (class_1)
0       1              0.32              0.68
1       1              0.24              0.76
2       0              1.00              0.00
3       0              0.93              0.07
4       0              0.93              0.07
